In [1]:
import numpy as np
from ultralytics import YOLO
import cv2
import cvzone
import math
from sort import *

model = YOLO("../Yolo-Weights/yolov8l.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


In [8]:
cap = cv2.VideoCapture("D:/study/CV/CK/Object-Detection-101/Videos/cars.mp4")  # For Video

mask = cv2.imread("D:/study/CV/CK/Object-Detection-101/Project 1 - Car Counter/mask.png")

In [10]:
# Khởi tạo đối tượng tracker với các tham số
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Định nghĩa giới hạn theo tọa độ [x1, y1, x2, y2]
limits = [400, 297, 673, 297]
totalCount = []  # Danh sách lưu trữ các đối tượng đã đếm

while True:
    # Đọc frame từ camera
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)

    # Gọi mô hình để nhận diện đối tượng trong vùng quan tâm
    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Lấy thông tin về hộp giới hạn (bounding box)
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Lấy độ tin cậy (confidence) và tên lớp của đối tượng
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Lọc ra các đối tượng là xe và có độ tin cậy lớn hơn 0.3
            if currentClass in ["car", "truck", "bus", "motorbike"] and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Cập nhật thông tin tracking của đối tượng
    resultsTracker = tracker.update(detections)

    # Vẽ đường giới hạn trên ảnh
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)

    # Hiển thị và đếm các đối tượng theo ID
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1

        # Vẽ khung và ID của đối tượng
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)

        # Vẽ điểm trung tâm của đối tượng
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # Kiểm tra xem đối tượng có vượt quá giới hạn không
        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)

    # Hiển thị số lượng đối tượng đã đếm
    cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    cv2.putText(img, str(len(totalCount)), (255, 100), cv2.FONT_HERSHEY_PLAIN, 5, (50, 50, 255), 8)

    # Hiển thị ảnh và chờ nhấn phím
    cv2.imshow("Image", img)
    cv2.waitKey(1)



0: 384x640 6 cars, 567.2ms
Speed: 5.7ms preprocess, 567.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 587.5ms
Speed: 1.2ms preprocess, 587.5ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 593.6ms
Speed: 2.2ms preprocess, 593.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 521.5ms
Speed: 2.1ms preprocess, 521.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 516.3ms
Speed: 2.8ms preprocess, 516.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 534.8ms
Speed: 4.8ms preprocess, 534.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 497.4ms
Speed: 2.0ms preprocess, 497.4ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 506.9ms
Speed: 2.3ms preprocess, 506.9ms inference, 0.0ms postprocess per image

KeyboardInterrupt: 

In [ ]:
import cv2
import cvzone
import math
import numpy as np
from sort import Sort

# Khởi tạo đối tượng tracker với các tham số
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Danh sách lưu trữ các đối tượng đã đếm
totalCount = []

# Đọc toàn bộ video
while True:
    success, img = cap.read()

    if not success:
        break  # Kết thúc nếu hết video

    imgRegion = cv2.bitwise_and(img, mask)

    # Gọi mô hình để nhận diện đối tượng trong vùng quan tâm
    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Lấy độ tin cậy và tên lớp của đối tượng
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Lọc ra các đối tượng là xe và có độ tin cậy lớn hơn 0.3
            if currentClass in ["car", "truck", "bus", "motorbike"] and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

# Cập nhật thông tin tracking của đối tượng sau khi đã xử lý toàn bộ video
resultsTracker = tracker.update(detections)

# Hiển thị và đếm các đối tượng theo ID
for result in resultsTracker:
    x1, y1, x2, y2, id = result
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    w, h = x2 - x1, y2 - y1

    # Vẽ khung và ID của đối tượng
    cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
    cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                       scale=2, thickness=3, offset=10)

    # Vẽ điểm trung tâm của đối tượng
    cx, cy = x1 + w // 2, y1 + h // 2
    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

    # Kiểm tra xem đối tượng có vượt quá giới hạn không
    if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
        if totalCount.count(id) == 0:
            totalCount.append(id)
            cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)

# Hiển thị số lượng đối tượng đã đếm
cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
cv2.putText(img, str(len(totalCount)), (255, 100), cv2.FONT_HERSHEY_PLAIN, 5, (50, 50, 255), 8)

# Hiển thị ảnh và chờ nhấn phím
cv2.imshow("Image", img)
cv2.waitKey(0)

# Đóng cửa sổ khi kết thúc
cv2.destroyAllWindows()



0: 384x640 4 cars, 502.5ms
Speed: 1.0ms preprocess, 502.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 545.4ms
Speed: 10.5ms preprocess, 545.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 703.1ms
Speed: 2.1ms preprocess, 703.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 523.8ms
Speed: 2.3ms preprocess, 523.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 524.8ms
Speed: 0.0ms preprocess, 524.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 514.3ms
Speed: 1.0ms preprocess, 514.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 507.3ms
Speed: 0.0ms preprocess, 507.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 506.8ms
Speed: 3.7ms preprocess, 5

KeyboardInterrupt: 